# PLEIADES Logging System Tutorial

This notebook demonstrates how to effectively use the logging system in PLEIADES, which is built on the [Loguru](https://github.com/Delgan/loguru) package.
We'll cover the basics of the logging system and how to customize it to suit your needs.

## 1. Basic Usage: Default Logging

PLEIADES automatically sets up a logging system when you import any of its modules. Let's start by importing PLEIADES and seeing the default logging behavior.

In [1]:
import pleiades

# Let's import a module that uses logging
from pleiades.nuclear.manager import NuclearDataManager

# Create a nuclear data manager, which will log its initialization
manager = NuclearDataManager()

When you ran the cell above, you should have seen some log output in the cell output. This is because PLEIADES configures a default logger that writes to both the console (stderr) and a log file.

By default:
- Console logging level is set to DEBUG
- Log files are stored in a `pleiades_logs` directory in your current working directory
- Log files are named with a timestamp pattern: `pleiades_YYYYMMDD_HH.log`
- Log files are rotated when they reach 10 MB and retained for 30 days

Let's see what log files have been created:

In [2]:
import os
from pathlib import Path

logs_dir = Path(os.getcwd()) / "pleiades_logs"
list(logs_dir.glob("*.log"))

[PosixPath('/Users/8cz/github.com/PLEIADES/examples/Notebooks/pleiades_logs/pleiades_20250428_18.log')]

## 2. Using the Logger Directly

PLEIADES provides two main ways to use the logging system:

1. Using the `loguru_logger` directly with the `bind` method
2. Using the `Logger` class for backward compatibility

In [3]:
# Method 1: Using loguru_logger directly
from pleiades.utils.logger import loguru_logger

# Bind a name to the logger - typically the module name
logger = loguru_logger.bind(name="my_notebook")

# Now we can log messages at different levels
logger.debug("This is a debug message")
logger.info("This is an info message")
logger.warning("This is a warning message")
logger.error("This is an error message")
logger.critical("This is a critical message")

2025-04-28 18:21:23 | DEBUG    | __main__:<module>:8 - This is a debug message
2025-04-28 18:21:23 | INFO     | __main__:<module>:9 - This is an info message
2025-04-28 18:21:23 | WARNING  | __main__:<module>:10 - This is a warning message
2025-04-28 18:21:23 | ERROR    | __main__:<module>:11 - This is an error message
2025-04-28 18:21:23 | CRITICAL | __main__:<module>:12 - This is a critical message


In [4]:
# Method 2: Using the Logger class (backward-compatible wrapper)
from pleiades.utils.logger import Logger

# Create a logger with a specific name
legacy_logger = Logger("legacy_logger")

# Use the same familiar methods
legacy_logger.debug("This is a debug message from the legacy logger")
legacy_logger.info("This is an info message from the legacy logger")
legacy_logger.warning("This is a warning message from the legacy logger")
legacy_logger.error("This is an error message from the legacy logger")
legacy_logger.critical("This is a critical message from the legacy logger")

2025-04-28 18:21:23 | INFO     | pleiades.utils.logger:__init__:145 - logging initialized...
2025-04-28 18:21:23 | DEBUG    | pleiades.utils.logger:debug:154 - This is a debug message from the legacy logger
2025-04-28 18:21:23 | INFO     | pleiades.utils.logger:info:163 - This is an info message from the legacy logger
2025-04-28 18:21:23 | WARNING  | pleiades.utils.logger:warning:172 - This is a warning message from the legacy logger
2025-04-28 18:21:23 | ERROR    | pleiades.utils.logger:error:181 - This is an error message from the legacy logger
2025-04-28 18:21:23 | CRITICAL | pleiades.utils.logger:critical:190 - This is a critical message from the legacy logger


## 3. Configuring the Logging System

PLEIADES provides a `configure_logger` function that allows you to customize the logging behavior. Let's see how to use it.

In [5]:
from pleiades.utils.logger import configure_logger

# Configure the logger to show only INFO and above messages to the console
# and add a custom log file
configure_logger(
    console_level="INFO",  # Show only INFO and above to console
    file_level="DEBUG",    # Still log all DEBUG and above to file
    log_file="my_custom_log.log"  # Use a custom log file
)

2025-04-28 18:21:23 | INFO     | pleiades.utils.logger:configure_logger:103 - Logging configured. Log file: /Users/8cz/github.com/PLEIADES/examples/Notebooks/pleiades_logs/my_custom_log.log


In [6]:
# Now let's log some messages to see the effect
logger = loguru_logger.bind(name="config_test")
logger.debug("This debug message won't show in the console, but will be in the log file")
logger.info("This info message will show both in the console and the log file")
logger.warning("This warning message will show in both places too")

2025-04-28 18:21:23 | INFO     | __main__:<module>:4 - This info message will show both in the console and the log file
2025-04-28 18:21:23 | WARNING  | __main__:<module>:5 - This warning message will show in both places too


Let's check the content of our custom log file:

In [7]:
# Check if our custom log file was created in the logs directory
custom_log_path = logs_dir / "my_custom_log.log"
print(f"Custom log exists: {custom_log_path.exists()}")

# View the last few lines of the log file
if custom_log_path.exists():
    # Read and print the last 10 lines
    with open(custom_log_path, 'r') as f:
        lines = f.readlines()
        print("Last 10 log entries:")
        for line in lines[-10:]:
            print(line.strip())

Custom log exists: True
Last 10 log entries:
2025-04-28 18:21:23 | INFO     | pleiades.utils.logger:configure_logger:103 - Logging configured. Log file: /Users/8cz/github.com/PLEIADES/examples/Notebooks/pleiades_logs/my_custom_log.log
2025-04-28 18:21:23 | DEBUG    | __main__:<module>:3 - This debug message won't show in the console, but will be in the log file
2025-04-28 18:21:23 | INFO     | __main__:<module>:4 - This info message will show both in the console and the log file
2025-04-28 18:21:23 | WARNING  | __main__:<module>:5 - This warning message will show in both places too


## 4. Advanced Configuration

Let's explore more advanced configuration options.

In [8]:
# Configure with custom rotation and retention policies
configure_logger(
    console_level="INFO",
    file_level="DEBUG",
    log_file="advanced_log.log",
    rotation="1 day",     # Rotate log files daily
    retention="1 week",   # Keep logs for one week
    format_string="{time:YYYY-MM-DD HH:mm:ss} | {level} | {name} - {message}"  # Custom format
)

2025-04-28 18:21:23 | INFO | pleiades.utils.logger - Logging configured. Log file: /Users/8cz/github.com/PLEIADES/examples/Notebooks/pleiades_logs/advanced_log.log


In [9]:
# Let's log some messages with our new configuration
logger = loguru_logger.bind(name="advanced_config")
logger.info("This message uses the custom format")
logger.warning("Another message with custom format")

2025-04-28 18:21:23 | INFO | __main__ - This message uses the custom format
2025-04-28 18:21:23 | WARNING | __main__ - Another message with custom format


## 5. Creating Module-Specific Loggers

In larger applications or scripts, it's a good practice to create dedicated loggers for different modules. This helps with organization and filtering.

In [10]:
# Let's create loggers for different components of our application
data_logger = loguru_logger.bind(name="data_processing")
analysis_logger = loguru_logger.bind(name="data_analysis")
ui_logger = loguru_logger.bind(name="user_interface")

# Now we can log from different components
data_logger.info("Loading data files...")
analysis_logger.info("Performing analysis on loaded data")
ui_logger.info("Displaying results to user")

# If an error occurs in a specific component
data_logger.error("Failed to load data file: file_xyz.dat")

# The errors are clearly associated with the specific component

2025-04-28 18:21:23 | INFO | __main__ - Loading data files...
2025-04-28 18:21:23 | INFO | __main__ - Performing analysis on loaded data
2025-04-28 18:21:23 | INFO | __main__ - Displaying results to user
2025-04-28 18:21:23 | ERROR | __main__ - Failed to load data file: file_xyz.dat


## 6. Creating Class-Based Loggers

For object-oriented code, you might want to have a logger per class instance. Let's see how to do that.

In [11]:
class DataProcessor:
    def __init__(self, name):
        self.name = name
        # Create a logger with both the class name and instance name
        self.logger = loguru_logger.bind(name=f"{self.__class__.__name__}.{name}")
    
    def process(self, data):
        self.logger.info(f"Processing {len(data)} data points")
        # Process data...
        self.logger.info(f"Finished processing")
        return data

# Create multiple instances, each with their own logger name
processor1 = DataProcessor("neutron_data")
processor2 = DataProcessor("gamma_data")

# Now when they log, we can distinguish between them
processor1.process([1, 2, 3, 4, 5])
processor2.process([10, 20, 30])

2025-04-28 18:21:23 | INFO | __main__ - Processing 5 data points
2025-04-28 18:21:23 | INFO | __main__ - Finished processing
2025-04-28 18:21:23 | INFO | __main__ - Processing 3 data points
2025-04-28 18:21:23 | INFO | __main__ - Finished processing


[10, 20, 30]

## 7. Practical Example: Tracking Operations in a PLEIADES Workflow

Let's put everything together in a practical example that demonstrates how logging can help track operations in a PLEIADES workflow.

In [12]:
# Configure logging for our workflow
configure_logger(
    console_level="INFO",
    file_level="DEBUG",
    log_file="workflow_example.log"
)

workflow_logger = loguru_logger.bind(name="workflow.example")
workflow_logger.info("Starting example workflow")

2025-04-28 18:21:23 | INFO     | pleiades.utils.logger:configure_logger:103 - Logging configured. Log file: /Users/8cz/github.com/PLEIADES/examples/Notebooks/pleiades_logs/workflow_example.log
2025-04-28 18:21:23 | INFO     | __main__:<module>:9 - Starting example workflow


In [13]:
# Import required modules
from pleiades.nuclear.manager import NuclearDataManager
from pleiades.nuclear.isotopes.manager import IsotopeManager

# Initialize managers
workflow_logger.info("Initializing managers...")
isotope_manager = IsotopeManager()
nuclear_manager = NuclearDataManager(isotope_manager=isotope_manager)

# Get information about an isotope
try:
    workflow_logger.info("Looking up isotope: Si-28")
    isotope_info = isotope_manager.get_isotope_info("Si-28")
    workflow_logger.info(f"Found isotope: {isotope_info}")
    
    # Create isotope parameters
    workflow_logger.info("Creating isotope parameters")
    isotope_params = nuclear_manager.create_isotope_parameters_from_string("Si-28")
    workflow_logger.info(f"Created parameters with library: {isotope_params.endf_library}")
    
    # This would download data in a real application
    # We'll just simulate it here to avoid making actual downloads
    workflow_logger.info("Would download ENDF data here...")
    
except Exception as e:
    workflow_logger.error(f"Workflow failed: {str(e)}")
    # In a real application, you might want to log the full traceback
    import traceback
    workflow_logger.error(f"Traceback: {traceback.format_exc()}")
else:
    workflow_logger.info("Workflow completed successfully")

2025-04-28 18:21:23 | INFO     | __main__:<module>:6 - Initializing managers...
2025-04-28 18:21:23 | INFO     | __main__:<module>:12 - Looking up isotope: Si-28
2025-04-28 18:21:23 | INFO     | __main__:<module>:14 - Found isotope: IsotopeInfo class for: Si-28
2025-04-28 18:21:23 | INFO     | __main__:<module>:17 - Creating isotope parameters
2025-04-28 18:21:23 | INFO     | __main__:<module>:19 - Created parameters with library: EndfLibrary.ENDF_B_VIII_0
2025-04-28 18:21:23 | INFO     | __main__:<module>:23 - Would download ENDF data here...
2025-04-28 18:21:23 | INFO     | __main__:<module>:31 - Workflow completed successfully


Searching for mass.mas20 in cached files for FileCategory.ISOTOPES: {PosixPath('/Users/8cz/github.com/PLEIADES/src/pleiades/nuclear/isotopes/files/neutrons.list'), PosixPath('/Users/8cz/github.com/PLEIADES/src/pleiades/nuclear/isotopes/files/isotopes.info'), PosixPath('/Users/8cz/github.com/PLEIADES/src/pleiades/nuclear/isotopes/files/mass.mas20')}
Checking file: neutrons.list
Checking file: isotopes.info
Checking file: mass.mas20
Found file: /Users/8cz/github.com/PLEIADES/src/pleiades/nuclear/isotopes/files/mass.mas20
Searching for isotopes.info in cached files for FileCategory.ISOTOPES: {PosixPath('/Users/8cz/github.com/PLEIADES/src/pleiades/nuclear/isotopes/files/neutrons.list'), PosixPath('/Users/8cz/github.com/PLEIADES/src/pleiades/nuclear/isotopes/files/isotopes.info'), PosixPath('/Users/8cz/github.com/PLEIADES/src/pleiades/nuclear/isotopes/files/mass.mas20')}
Checking file: neutrons.list
Checking file: isotopes.info
Found file: /Users/8cz/github.com/PLEIADES/src/pleiades/nuclear

## 8. Summary

In this tutorial, we've covered:

1. **Basic logging** with PLEIADES' default configuration
2. **Different ways to use the logger**:
   - Using `loguru_logger` directly with the `bind` method
   - Using the backwards-compatible `Logger` class
3. **Custom configuration** with `configure_logger`
4. **Advanced configuration** options like custom formats and rotation policies
5. **Module-specific loggers** for better organization
6. **Class-based loggers** for object-oriented code
7. **A practical workflow example**

The PLEIADES logging system, built on Loguru, provides a flexible and powerful way to track operations, debug issues, and maintain a record of your application's behavior. By properly utilizing the logging system, you can make your code more maintainable and easier to troubleshoot.